In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy.stats as stats
from scipy.signal import argrelextrema
%matplotlib inline
import math
import seaborn as sns
import statsmodels.api as sm 
from patsy import dmatrices 

**Importing Data/fixing length discrepancy/Converting to same timezone**

In [2]:
#Pull significant events data
sig_events_df = pd.read_csv('../Data/GDELT_data/article_counts_disasters.csv', index_col = 'datetime')
stock_df = pd.read_csv('../Data/stock_prices_cleaned.csv', index_col = 'Date')
#Convert datetime columns to appropraite dtype
sig_events_df.index = pd.to_datetime(sig_events_df.index).floor('D')
stock_df.index = pd.to_datetime(stock_df.index, utc=True).floor('D')
#Drop duplicate index column
sig_events_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
stock_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
#Convert stock data to UTC
print(sig_events_df)
print(stock_df)

                           All Articles  MANMADE_DISASTER_IMPLIED  \
datetime                                                            
2017-01-01 00:00:00+00:00        427906                     87559   
2017-01-02 00:00:00+00:00        592774                    109720   
2017-01-03 00:00:00+00:00        757916                    129371   
2017-01-04 00:00:00+00:00        822862                    134633   
2017-01-05 00:00:00+00:00        830028                    143110   
...                                 ...                       ...   
2022-11-06 00:00:00+00:00        166754                     30718   
2022-11-07 00:00:00+00:00        281263                     43567   
2022-11-08 00:00:00+00:00        306702                     44237   
2022-11-09 00:00:00+00:00        327495                     45854   
2022-11-10 00:00:00+00:00        288809                     43763   

                           WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE  \
datetime    

In [3]:
#Just going to drop these values to ensure dataframes are the same length/dates
print(list(set(stock_df.index) - set(sig_events_df.index)))
stock_df.drop(list(set(stock_df.index) - set(sig_events_df.index)), inplace=True)
print(len(stock_df))
#Only keep columns from GDELT that were identified with high correlation in initial analysis
#Merge Dataframes
merged_df = sig_events_df.join(stock_df, how='inner')

[Timestamp('2017-12-02 00:00:00+0000', tz='UTC'), Timestamp('2020-10-20 00:00:00+0000', tz='UTC'), Timestamp('2017-12-01 00:00:00+0000', tz='UTC')]
2137


In [21]:
#Find R2 for all tickers using MLR 
mlr_df = pd.DataFrame(np.nan,index = [0], columns = stock_df.columns)
for ticker in stock_df.columns:
    #Generate string for MLR. Exclude the all articles column
    of_interest_events = sig_events_df.loc[:, sig_events_df.columns != 'All Articles']
    dmatrices_str = ticker + ' ~ ' 
    for i, col in enumerate(of_interest_events.columns):
        if i < len(of_interest_events.columns)-1:
            dmatrices_str += col + '+'
        else:
            dmatrices_str += col

    #MLR generation
    y_2_poly, X_2_poly = dmatrices(dmatrices_str, data=merged_df, return_type='dataframe')
    mlr_2 =sm.OLS(y_2_poly, X_2_poly).fit()
    #Save R^2 value
    mlr_df[ticker] = mlr_2.rsquared
    
mlr_df

,SHEL,CVX,BP,XOM,XEL,TSLA,IBDRY,JKS
0,0.668287,0.552996,0.676086,0.585151,0.837638,0.706229,0.810698,0.651146
